In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# Install the psycopg2 package
%pip install psycopg2-binary
import psycopg2



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
base_path = r'F:\GUVI\Projects\P2 -Data Spark'

sales = pd.read_csv(os.path.join(base_path, 'sales.csv'))
customers = pd.read_csv(os.path.join(base_path, 'customers.csv'), encoding='ISO 8859-1')
products = pd.read_csv(os.path.join(base_path, 'products.csv'))
exchange_rates = pd.read_csv(os.path.join(base_path, 'exchange_rates.csv'))
stores = pd.read_csv(os.path.join(base_path, 'stores.csv'))


In [3]:
# Apply the operation on column names for each DataFrame in the dictionary
dataframes = {
    'sales': sales,
    'customers': customers,
    'products': products,
    'exchange_rates': exchange_rates,
    'stores': stores
}
for name, df in dataframes.items():
    if isinstance(df, pd.DataFrame):  # Ensure it's a DataFrame
        # Perform column name transformation: lowercase and replace spaces with underscores
        df.columns = df.columns.str.lower().str.replace(' ', '_')
        print(f"Columns for '{name}' after transformation:")
        print(df.columns, "\n")


Columns for 'sales' after transformation:
Index(['order_number', 'line_item', 'order_date', 'delivery_date',
       'customerkey', 'storekey', 'productkey', 'quantity', 'currency_code'],
      dtype='object') 

Columns for 'customers' after transformation:
Index(['customerkey', 'gender', 'name', 'city', 'state_code', 'state',
       'zip_code', 'country', 'continent', 'birthday'],
      dtype='object') 

Columns for 'products' after transformation:
Index(['productkey', 'product_name', 'brand', 'color', 'unit_cost_usd',
       'unit_price_usd', 'subcategorykey', 'subcategory', 'categorykey',
       'category'],
      dtype='object') 

Columns for 'exchange_rates' after transformation:
Index(['date', 'currency', 'exchange'], dtype='object') 

Columns for 'stores' after transformation:
Index(['storekey', 'country', 'state', 'square_meters', 'open_date'], dtype='object') 



In [4]:
# List of dataframes to check for null values and data types
dataframes = {
    'sales': sales,
    'customers': customers,
    'products': products,
    'exchange_rates': exchange_rates,
    'stores': stores
}

for name, df in dataframes.items():
    print(f"--- {name.upper()} ---")
    print(f"Null values per column:\n{df.isnull().sum()}")
    print("\nData types:")
    print(df.dtypes)
    print("\n")


--- SALES ---
Null values per column:
order_number         0
line_item            0
order_date           0
delivery_date    49719
customerkey          0
storekey             0
productkey           0
quantity             0
currency_code        0
dtype: int64

Data types:
order_number      int64
line_item         int64
order_date       object
delivery_date    object
customerkey       int64
storekey          int64
productkey        int64
quantity          int64
currency_code    object
dtype: object


--- CUSTOMERS ---
Null values per column:
customerkey     0
gender          0
name            0
city            0
state_code     10
state           0
zip_code        0
country         0
continent       0
birthday        0
dtype: int64

Data types:
customerkey     int64
gender         object
name           object
city           object
state_code     object
state          object
zip_code       object
country        object
continent      object
birthday       object
dtype: object


--- PRODUCTS 

In [5]:
# Loop through each dataframe to find the number of columns, rows, and column names
for name, df in dataframes.items():
    rows, columns = df.shape  
    column_names = df.columns.tolist()  
    print(f"--- {name.upper()} ---")
    print(f"Number of rows: {rows}")
    print(f"Number of columns: {columns}")
    print(f"Column names: {', '.join(column_names)}\n")  



--- SALES ---
Number of rows: 62884
Number of columns: 9
Column names: order_number, line_item, order_date, delivery_date, customerkey, storekey, productkey, quantity, currency_code

--- CUSTOMERS ---
Number of rows: 15266
Number of columns: 10
Column names: customerkey, gender, name, city, state_code, state, zip_code, country, continent, birthday

--- PRODUCTS ---
Number of rows: 2517
Number of columns: 10
Column names: productkey, product_name, brand, color, unit_cost_usd, unit_price_usd, subcategorykey, subcategory, categorykey, category

--- EXCHANGE_RATES ---
Number of rows: 11215
Number of columns: 3
Column names: date, currency, exchange

--- STORES ---
Number of rows: 67
Number of columns: 5
Column names: storekey, country, state, square_meters, open_date



In [6]:
sales.isnull().sum()

order_number         0
line_item            0
order_date           0
delivery_date    49719
customerkey          0
storekey             0
productkey           0
quantity             0
currency_code        0
dtype: int64

In [7]:
# # Iterate through each DataFrame and find the number of empty (NaN) cells
# for name, df in dataframes.items():
#     if isinstance(df, pd.DataFrame):  # Ensure it's a DataFrame
#         # Count the number of empty (NaN) cells in the entire DataFrame
#         empty_cells = df.isnull().sum().sum()
#         print(f"Number of empty cells in '{name}': {empty_cells}")


In [8]:
# # Iterate through each DataFrame and find null values
# for name, df in dataframes.items():
#     if isinstance(df, pd.DataFrame):  # Ensure it's a DataFrame
#         print(f"Null values in '{name}':")
#         print(df.isnull().sum(), "\n")  # Display null counts for each column


In [9]:
# Using .loc[] 
sales['order_date'] = pd.to_datetime(sales['order_date'], format='%m/%d/%Y')
sales['order_date'] = sales['order_date'].dt.date
sales['delivery_date'] = pd.to_datetime(sales['delivery_date'], format='%m/%d/%Y', errors='coerce')
sales['delivery_date'] = sales['delivery_date'].dt.date
sales.loc[:, 'currency_code'] = sales['currency_code'].astype(str)


In [10]:
stores['country'] = stores['country'].astype(str)
stores['state'] = stores['state'].astype(str)
stores['open_date'] = pd.to_datetime(stores['open_date'], format='%m/%d/%Y', errors='coerce')
stores['open_date'] = stores['open_date'].dt.date

In [11]:
# Remove dollar signs and commas, then convert to float
products['unit_cost_usd'] = products['unit_cost_usd'].replace(r'[\$,]', '', regex=True).astype(float)
products['unit_price_usd'] = products['unit_price_usd'].replace(r'[\$,]', '', regex=True).astype(float)


In [12]:
customers['birthday'] = pd.to_datetime(customers['birthday'], format='%m/%d/%Y', errors='coerce')
customers['birthday'] = customers['birthday'].dt.date

In [13]:
exchange_rates['date'] = pd.to_datetime(exchange_rates['date'], format='%m/%d/%Y', errors='coerce')
exchange_rates['date'] = exchange_rates['date'].dt.date

In [14]:
# Over_all = pd.merge(customers, sales1, on='CustomerKey', how='inner')
# Over_all = pd.merge(Over_all, exchange_rates, left_on=['Order Date', 'Currency Code'], right_on=['Date', 'Currency'], how = 'inner')
# Over_all = pd.merge(Over_all,products,on ='ProductKey', how ='inner')
# Over_all = pd.merge(Over_all,stores, on =['StoreKey', 'Country', 'State'])

In [15]:
sales.head()

,order_number,line_item,order_date,delivery_date,customerkey,storekey,productkey,quantity,currency_code
0,366000,1,2016-01-01,NaT,265598,10,1304,1,CAD
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD


In [16]:
# Calculate the difference in days
sales['days_difference'] = sales.apply(
    lambda row: (row['delivery_date'] - row['order_date']).days if pd.notnull(row['order_date']) and pd.notnull(row['delivery_date']) else 0,
    axis=1
)

In [17]:
sales.describe()

,order_number,line_item,customerkey,storekey,productkey,quantity,days_difference
count,6.288400e+04,62884.000000,6.288400e+04,62884.000000,62884.000000,62884.000000,62884.000000
mean,1.430905e+06,2.164207,1.180797e+06,31.802144,1125.859344,3.144790,0.948190
std,4.532963e+05,1.365170,5.859634e+05,22.978188,709.244010,2.256371,2.082614
min,3.660000e+05,1.000000,3.010000e+02,0.000000,1.000000,1.000000,0.000000
25%,1.121017e+06,1.000000,6.808580e+05,8.000000,437.000000,1.000000,0.000000
50%,1.498016e+06,2.000000,1.261200e+06,37.000000,1358.000000,2.000000,0.000000
75%,1.788010e+06,3.000000,1.686496e+06,53.000000,1650.000000,4.000000,0.000000
max,2.243032e+06,7.000000,2.099937e+06,66.000000,2517.000000,10.000000,17.000000


In [18]:
days_difference_mean = sales['days_difference'].mean()
print (days_difference_mean)

0.9481903186820176


In [19]:
days_difference_mean_no_zeros = sales[sales['days_difference'] != 0]['days_difference'].mean()
print(days_difference_mean_no_zeros)

4.529130269654386


In [20]:
# Create the revised_delivery_date column
sales['revised_delivery_date'] = sales.apply(
    lambda row: row['delivery_date'] if pd.notnull(row['delivery_date']) else row['order_date'] + pd.Timedelta(days=5),
    axis=1
)



In [21]:
sales.head()

,order_number,line_item,order_date,delivery_date,customerkey,storekey,productkey,quantity,currency_code,days_difference,revised_delivery_date
0,366000,1,2016-01-01,NaT,265598,10,1304,1,CAD,0,2016-01-06
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD,12,2016-01-13
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD,12,2016-01-13
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD,11,2016-01-12
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD,11,2016-01-12


In [22]:
# sales.to_excel(os.path.join(base_path, "revised_sales.xlsx"), index=False)

In [23]:
sales_1 = sales

In [24]:
sales_1 = sales_1.drop(['delivery_date', 'days_difference'], axis=1)



In [25]:
sales_1.head()

,order_number,line_item,order_date,customerkey,storekey,productkey,quantity,currency_code,revised_delivery_date
0,366000,1,2016-01-01,265598,10,1304,1,CAD,2016-01-06
1,366001,1,2016-01-01,1269051,0,1048,2,USD,2016-01-13
2,366001,2,2016-01-01,1269051,0,2007,1,USD,2016-01-13
3,366002,1,2016-01-01,266019,0,1106,7,CAD,2016-01-12
4,366002,2,2016-01-01,266019,0,373,1,CAD,2016-01-12


In [26]:
Over_all = pd.merge(customers, sales_1, on='customerkey', how='inner')

In [27]:
Over_all.shape

(62884, 18)

In [28]:
Over_all = pd.merge(Over_all, exchange_rates, left_on=['order_date', 'currency_code'], right_on=['date', 'currency'], how = 'inner')

In [29]:
Over_all.shape

(62884, 21)

In [30]:
Over_all = pd.merge(Over_all,products,on ='productkey', how ='inner')

In [31]:
Over_all.shape

(62884, 30)

In [32]:
Over_all = pd.merge(Over_all,stores, on =['storekey', 'country', 'state'])


In [33]:
Over_all.shape

(6181, 32)

In [34]:
Over_all.fillna(0)

,customerkey,gender,name,city,state_code,state,zip_code,country,continent,birthday,...,brand,color,unit_cost_usd,unit_price_usd,subcategorykey,subcategory,categorykey,category,square_meters,open_date
0,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27,...,Wide World Importers,Silver,160.49,349.00,303,Desktops,3,Computers,2000.0,2010-01-01
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27,...,Proseware,Black,72.66,158.00,306,"Printers, Scanners & Fax",3,Computers,2000.0,2010-01-01
2,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27,...,Contoso,Grey,214.03,646.00,402,Digital SLR Cameras,4,Cameras and camcorders,2000.0,2010-01-01
3,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27,...,Adventure Works,Silver,137.63,269.95,303,Desktops,3,Computers,2000.0,2010-01-01
4,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27,...,Northwind Traders,White,17.45,37.95,106,Bluetooth Headphones,1,Audio,2000.0,2010-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6176,2097018,Female,Thérèse Lapresse,Simpsonville,SC,South Carolina,29681,United States,North America,1968-05-29,...,Wide World Importers,White,257.06,559.00,303,Desktops,3,Computers,2000.0,2012-12-15
6177,2098404,Female,Lenka Cadilová,Pump Station Three,AK,Alaska,99701,United States,North America,1971-12-02,...,Contoso,White,61.62,134.00,101,MP4&MP3,1,Audio,1190.0,2015-01-01
6178,2098404,Female,Lenka Cadilová,Pump Station Three,AK,Alaska,99701,United States,North America,1971-12-02,...,A. Datum,Pink,66.26,200.00,401,Digital Cameras,4,Cameras and camcorders,1190.0,2015-01-01
6179,2098404,Female,Lenka Cadilová,Pump Station Three,AK,Alaska,99701,United States,North America,1971-12-02,...,Contoso,Black,116.75,229.00,305,Projectors & Screens,3,Computers,1190.0,2015-01-01


In [35]:
Over_all.columns

Index(['customerkey', 'gender', 'name', 'city', 'state_code', 'state',
       'zip_code', 'country', 'continent', 'birthday', 'order_number',
       'line_item', 'order_date', 'storekey', 'productkey', 'quantity',
       'currency_code', 'revised_delivery_date', 'date', 'currency',
       'exchange', 'product_name', 'brand', 'color', 'unit_cost_usd',
       'unit_price_usd', 'subcategorykey', 'subcategory', 'categorykey',
       'category', 'square_meters', 'open_date'],
      dtype='object')

In [36]:
print(Over_all.dtypes)


customerkey                int64
gender                    object
name                      object
city                      object
state_code                object
state                     object
zip_code                  object
country                   object
continent                 object
birthday                  object
order_number               int64
line_item                  int64
order_date                object
storekey                   int64
productkey                 int64
quantity                   int64
currency_code             object
revised_delivery_date     object
date                      object
currency                  object
exchange                 float64
product_name              object
brand                     object
color                     object
unit_cost_usd            float64
unit_price_usd           float64
subcategorykey             int64
subcategory               object
categorykey                int64
category                  object
square_met

In [38]:
import pymysql

connection = pymysql.connect(host='localhost', user='root', password='root', port=3306)

try:
    cursor = connection.cursor()

    database_name = "DataSpark"
    cursor.execute(f"CREATE DATABASE {database_name}")
    print(f"Database {database_name} created successfully")

except pymysql.MySQLError as error:
    print(f"Error occurred: {error}")

finally:
    cursor.close()
    connection.close()


Database DataSpark created successfully


In [41]:
from sqlalchemy import create_engine

host='localhost'
user='root'
password='root'
port=3306
database_name = "DataSpark"

engine_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database_name}"
engine = create_engine(engine_string)

table_name ="over_all"
Over_all.to_sql(table_name, engine, if_exists='replace', index=False)
print(f"successfully imported {table_name} to sql")

table_name ="customers"
customers.to_sql(table_name, engine, if_exists='replace', index=False)
print(f"successfully imported {table_name} to sql")

table_name = "products"
products.to_sql(table_name, engine, if_exists='replace', index=False)
print(f"successfully imported {table_name} to sql")

table_name = "exchangerates"
exchange_rates.to_sql(table_name, engine, if_exists='replace', index=False)
print(f"successfully imported {table_name} to sql")

table_name = "stores"
stores.to_sql(table_name, engine, if_exists='replace', index=False)
print(f"successfully imported {table_name} to sql")


successfully imported over_all to sql
successfully imported customers to sql
successfully imported products to sql
successfully imported exchangerates to sql
successfully imported stores to sql


In [82]:
Over_all.columns

Index(['customerkey', 'gender', 'name', 'city', 'state_code', 'state',
       'zip_code', 'country', 'continent', 'birthday', 'order_number',
       'line_item', 'order_date', 'storekey', 'productkey', 'quantity',
       'currency_code', 'revised_delivery_date', 'date', 'currency',
       'exchange', 'product_name', 'brand', 'color', 'unit_cost_usd',
       'unit_price_usd', 'subcategorykey', 'subcategory', 'categorykey',
       'category', 'square_meters', 'open_date'],
      dtype='object')

In [83]:
# Set display option to show all columns
pd.set_option('display.max_columns', None)

# Print the first 5 rows of the DataFrame
print(Over_all.head())


   customerkey  gender          name        city state_code  \
0          325  Female  Madison Hull  MOUNT BUDD         WA   
1          325  Female  Madison Hull  MOUNT BUDD         WA   
2          325  Female  Madison Hull  MOUNT BUDD         WA   
3          325  Female  Madison Hull  MOUNT BUDD         WA   
4          325  Female  Madison Hull  MOUNT BUDD         WA   

               state zip_code    country  continent    birthday  order_number  \
0  Western Australia     6522  Australia  Australia  1979-09-27       1109001   
1  Western Australia     6522  Australia  Australia  1979-09-27       1109001   
2  Western Australia     6522  Australia  Australia  1979-09-27       1109001   
3  Western Australia     6522  Australia  Australia  1979-09-27       1781006   
4  Western Australia     6522  Australia  Australia  1979-09-27       1781006   

   line_item  order_date  storekey  productkey  quantity currency_code  \
0          1  2018-01-13         6         443         4    